In [1]:
import numpy as np
import os
from fine_tune_model import define_classifier_for_classification_for_response
from validation_method import stratified_5_fold_classification, nested_cross_validation_classification, loocv_classification
from utils_simple_prediction import add_task_change_data, load_data_for_classification, add_cgi, add_mddr, set_path, load_task_change_data, save_model_seed
import time
import pandas as pd
set_path()

"""

result[model][modality][...]


"""
models = ['Naive Bayes', 'Discriminant Analysis(LDA)', 'Random Forest', 'XGBoost', 'SVM', 'SGDClassifier']#['Discriminant Analysis(LDA)', 'Naive Bayes'] #  'KNN', 'MLP',  , 'Discriminant Analysis(QDA)'
modalities = ['fnirs_modality_classification', 'clinical_data_and_fnirs_modality_classification', 'clinical_data_modality_classification'] # ['fnirs_modality_classification'] # 
modalities_to_name = {
    'clinical_data_modality_classification': 'Tier 1', 
    'fnirs_modality_classification': 'Tier 2', 
    'clinical_data_and_fnirs_modality_classification': 'Tier 3'

}
main_fold = 'results/ML_results/simple_prediction_response/'


auc_result = result = {model: {modality: {} for modality in modalities} for model in models}


Current system is Ubuntu


In [ ]:
# for Transformer 

In [42]:
# Compare if it is better than chance
all_auc_fnirs = []
all_auc_fnirs_cli = []
all_bacc_fnirs_cli = []
all_bacc_cli = []

for model in models:
    if model == 'SGDClassifier':
        continue    
    for modality in modalities:
        result_pth = main_fold + modality + '/' + model + '.csv'
        df = pd.read_csv(result_pth)
        
        
        if modality == 'fnirs_modality_classification':
            all_auc_fnirs.append(df['test_auc'].tolist())
        
        if modality == 'clinical_data_and_fnirs_modality_classification':
            all_auc_fnirs_cli.append(df['test_auc'].tolist())

        
        if modality == 'clinical_data_modality_classification':
            all_bacc_cli.append(df['test_acc'].tolist())
        else:
            all_bacc_fnirs_cli.append(df['test_acc'].tolist())


all_auc_fnirs = np.array(all_auc_fnirs).reshape(-1).tolist()
all_auc_fnirs_cli = np.array(all_auc_fnirs_cli).reshape(-1).tolist()

In [43]:
from sklearn.metrics import roc_auc_score, roc_curve, auc

fnirs_cli_transformer_pth = 'results/gnn_transformer_with_cli_demo_v1/prognosis_mix_hb/pretreatment_response/ten-fold-evaluate/y_pred_test_all.npy'
fnirs_transformer_pth = 'results/gnn_transformer/prognosis_mix_hb/pretreatment_response/ten-fold-evaluate/y_pred_test_all.npy'
label_path = 'allData/prognosis_mix_hb/pretreatment_response/label.npy'

def add_into_all_auc(all_auc, MMDR_path, label_path):
    data = np.load(MMDR_path)
    y_true = np.load(label_path)
    for i in range(data.shape[0]):
        y_pred = data[i,:,1]
        fpr, tpr, _ = roc_curve(y_true, y_pred)

        roc_auc = auc(fpr, tpr)
        all_auc.append(roc_auc)
    print(all_auc)
    return all_auc

all_auc_fnirs = add_into_all_auc(all_auc_fnirs, fnirs_transformer_pth, label_path)
all_auc_fnirs_cli = add_into_all_auc(all_auc_fnirs_cli, fnirs_cli_transformer_pth, label_path)

[0.7524, 0.7456, 0.7823, 0.7701, 0.7483, 0.7551, 0.7673, 0.7442, 0.7619, 0.7878, 0.7986, 0.7565, 0.7646, 0.7592, 0.7864, 0.7456, 0.7456, 0.7401, 0.7565, 0.7646, 0.7946, 0.7959, 0.7633, 0.7687, 0.7864, 0.7578, 0.7442, 0.7986, 0.7483, 0.7592, 0.7878, 0.7837, 0.7442, 0.7429, 0.7592, 0.7714, 0.7401, 0.7456, 0.7946, 0.7456, 0.7537, 0.7878, 0.7918, 0.8027, 0.7796, 0.7701, 0.7741, 0.7633, 0.8014, 0.7633, 0.7633, 0.7565, 0.7864, 0.7456, 0.7442, 0.7456, 0.7918, 0.766, 0.7551, 0.7782, 0.766, 0.7796, 0.7769, 0.7537, 0.7633, 0.7537, 0.7701, 0.7619, 0.7578, 0.7633, 0.785, 0.7918, 0.7891, 0.7483, 0.7755, 0.7605, 0.7864, 0.7374, 0.7769, 0.7442, 0.7769, 0.751, 0.785, 0.7374, 0.7864, 0.7551, 0.7986, 0.7469, 0.7701, 0.7796, 0.7755, 0.7537, 0.7959, 0.7728, 0.7605, 0.7592, 0.7891, 0.7578, 0.7592, 0.7796, 0.6993, 0.717, 0.7061, 0.698, 0.7102, 0.7075, 0.7034, 0.7048, 0.7034, 0.7102, 0.6966, 0.698, 0.7034, 0.6966, 0.7211, 0.7034, 0.7102, 0.7129, 0.698, 0.7116, 0.702, 0.7061, 0.7184, 0.7116, 0.698, 0.7265, 0.

In [44]:

all_bacc_fnirs_cli = np.array(all_bacc_fnirs_cli).reshape(-1)
all_bacc_cli = np.array(all_bacc_cli).reshape(-1)

# all_auc_fnirs = np.array(all_auc_fnirs).reshape(-1)
# all_auc_fnirs_cli = np.array(all_auc_fnirs_cli).reshape(-1)


import numpy as np
from scipy.stats import binom_test

def test_classifier_performance(bAcc_scores, chance_level=0.5, alpha=0.05):
    """
    Perform a one-tailed binomial test to compare classifier performance to chance.

    Parameters:
    bAcc_scores (list or np.array): List or array of balanced accuracy scores of the classifiers.
    chance_level (float): The chance level performance (default is 0.5 for balanced accuracy).
    alpha (float): Significance level for the hypothesis test (default is 0.05).

    Returns:
    p_value (float): The p-value from the one-tailed binomial test.
    result (str): The result of the hypothesis test.
    """
    # Convert bAcc_scores to a numpy array if it's not already
    bAcc_scores = np.array(bAcc_scores)
    
    # Calculate the number of successes (i.e., scores greater than chance_level)
    successes = np.sum(bAcc_scores > chance_level)
    
    # Total number of trials
    n = len(bAcc_scores)
    
    # Perform one-tailed binomial test
    p_value = binom_test(successes, n, chance_level, alternative='greater')
    
    # Determine the result based on the p-value and significance level
    if p_value < alpha:
        result = f"Reject the null hypothesis. Classifier performance is significantly better than chance (p-value: {p_value:.4f})."
    else:
        result = f"Fail to reject the null hypothesis. No significant difference from chance (p-value: {p_value:.4f})."
    
    return p_value, result



def test_unimodal_vs_bimodal(unimodal_scores, bimodal_scores, alpha=0.05):
    """
    Perform a one-tailed sign test to compare unimodal and bimodal classifier performances.

    Parameters:
    unimodal_scores (list or np.array): List or array of AUC scores of the unimodal classifiers.
    bimodal_scores (list or np.array): List or array of AUC scores of the bimodal classifiers.
    alpha (float): Significance level for the hypothesis test (default is 0.05).

    Returns:
    p_value (float): The p-value from the one-tailed sign test.
    result (str): The result of the hypothesis test.
    """
    # Convert scores to numpy arrays if they are not already
    unimodal_scores = np.array(unimodal_scores)
    bimodal_scores = np.array(bimodal_scores)
    
    # Ensure the scores arrays have the same length
    if unimodal_scores.shape[0] != bimodal_scores.shape[0]:
        raise ValueError("The unimodal and bimodal scores must have the same length.")
    
    # Calculate the number of successes (i.e., bimodal scores greater than unimodal scores)
    successes = np.sum(unimodal_scores > bimodal_scores)
    
    # Total number of trials (excluding ties)
    n = np.sum(bimodal_scores != unimodal_scores)
    
    # Perform one-tailed sign test using binomial test
    p_value = binom_test(successes, n, 0.5, alternative='greater')
    
    # Determine the result based on the p-value and significance level
    if p_value < alpha:
        result = f"Reject the null hypothesis. unimodal classifiers significantly outperform bimodal classifiers (p-value: {p_value:.4f})."
    else:
        result = f"Fail to reject the null hypothesis. No significant difference in performance (p-value: {p_value:.4f})."
    
    return p_value, result


print("When comparing to chance")

print('Using fNIRS or fNIRS+Cli data', end='')
print(test_classifier_performance(all_bacc_fnirs_cli))

print('Using Cli data', end='')
print(test_classifier_performance(all_bacc_cli))



print("When comparing fnirs and fnirs+cli data")
test_unimodal_vs_bimodal(all_auc_fnirs, all_auc_fnirs_cli)

When comparing to chance
Using fNIRS or fNIRS+Cli data(1.0015559486427974e-166, 'Reject the null hypothesis. Classifier performance is significantly better than chance (p-value: 0.0000).')
Using Cli data(0.9822716809936585, 'Fail to reject the null hypothesis. No significant difference from chance (p-value: 0.9823).')
When comparing fnirs and fnirs+cli data


(6.571190906607724e-109,
 'Reject the null hypothesis. unimodal classifiers significantly outperform bimodal classifiers (p-value: 0.0000).')

: 